In [ ]:
# Import modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from warnings import filterwarnings
filterwarnings('ignore')

#plt.rcParams['figure.figsize'] = (20, 10)
#plt.rcParams['font.family'] = "serif"

In [ ]:
df = pd.read_csv('D:\\Study\\DataScience\\Data\\MachineLearningData\\HighwayData\\vartest.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['Yr'] = pd.to_datetime(df['Yr'], format = '%Y')
df = df.set_index('Yr')
df.index = pd.DatetimeIndex(df.index)

In [ ]:
df.head()

In [ ]:
#creating the train and validation set
train = df[:int(0.8*(len(df)))]
valid = df[int(0.8*(len(df))):]

In [ ]:
valid

In [ ]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

In [ ]:
model = VAR(endog=train)
model_fit = model.fit()

In [ ]:
# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
#converting predictions to dataframe
cols = df.columns
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])

In [ ]:
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,len(cols)):
    for i in range(0, len(prediction)):
        pred.iloc[i][j] = prediction[i][j]

In [ ]:
#check rmse
from sklearn.metrics import mean_squared_error

for i in range(0,len(cols)):
    #print(cols[i])
    mse = mean_squared_error(pred.iloc[:,i], valid.iloc[:,i])
    rmse = np.sqrt(mse)
    print('RMSE for', cols[i],'is:',format(np.round(rmse,3)))

In [ ]:
#make final predictions
model = VAR(endog=df)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

### VAR model with multiple highway segments

In [ ]:
data = pd.read_csv('D:\\Study\\DataScience\\Data\\MachineLearningData\\HighwayData\\varmultiple.csv')

In [ ]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR




districts = list(data['Maintenance District'].unique())

for district in districts:
    
    data_new = data[data['Maintenance District'] == district]
    segments = list(data_new['Segment'].unique())
    
    for segment in segments:
        print(district, segment)
        
        df = data[(data['Maintenance District'] == district) & (data['Segment'] == segment)]
        
        df['Year'] = pd.to_datetime(df['Year'], format = '%Y')
        df = df.set_index('Year')
        df.index = pd.DatetimeIndex(df.index)
        df = df.iloc[:, 5:]
        #print(df)
        #print(df.dtypes)
        model = VAR(endog=df)
        model_fit = model.fit()
        yhat = model_fit.forecast(model_fit.y, steps=1)
        print(yhat)
            

In [319]:

df_predict = pd.DataFrame(columns= ['Year', 'Route', 'Begin Mile', 'End Mile', 'Segment', 'Maintenance District'])

districts = list(data['Maintenance District'].unique())
predic = []

for district in districts:
   
    
    data_new = data[data['Maintenance District'] == district]
    segments = list(data_new['Segment'].unique())
    
    for segment in segments: 
        #print(district, segment)
        
        Route = data[(data['Maintenance District'] == district) & (data['Segment'] == segment)]['Route'].unique()[0]
        BeginMile = data[(data['Maintenance District'] == district) & (data['Segment'] == segment)]['Begin Mile'].unique()[0]
        EndMile = data[(data['Maintenance District'] == district) & (data['Segment'] == segment)]['End Mile'].unique()[0]
        Segment = segment
        
        
        df = data[(data['Maintenance District'] == district) & (data['Segment'] == segment)]
                
        
        df['Year'] = pd.to_datetime(df['Year'], format = '%Y')
        df = df.set_index('Year')
        df.index = pd.DatetimeIndex(df.index)
        df = df.iloc[:, 5:]
   
        model = VAR(endog=df)
        model_fit = model.fit()
        yhat = model_fit.forecast(model_fit.y, steps=1)
    
        predic.append(yhat[0])
        
        df_predict = df_predict.append({'Year': 2019, 'Route': Route, 'Begin Mile': BeginMile, 
                              'End Mile': EndMile, 'Segment': segment, 'Maintenance District': district}, ignore_index=True)
                   

In [320]:
df_predict

,Year,Route,Begin Mile,End Mile,Segment,Maintenance District
0,2019,BL-11-M,0.46,2.46,X1,1 - DEMING
1,2019,BL-11-M,2.46,4.46,X2,1 - DEMING
2,2019,BL-11-M,4.46,5.42,X3,1 - DEMING
3,2019,BL-11-P,0.00,0.35,X4,1 - DEMING
4,2019,US-84-M,0.00,0.22,X5,2 - ROSWELL


In [317]:
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(predic)),columns=df.columns)
for i in range(0,len(predic)):
    for j in range(0, len(predic[0])):
        pred.iloc[i][j] = predic[i][j]

In [318]:
pred

,Overall Condition Index (OCI),Structural Index,Environmental Index,Average IRI,Rutting Measure,Alligator Cracking WP Index,Edge Cracking Index,Long Cracking Index,Patch Deterioration Index,Raveling and Weathering Index,FW Index,Pavement Condition Rating (PCR)
0,60.4022,85.6472,88.9381,63.4825,0.143771,104.801,88.542,83.1232,85.4547,72.1856,81.9556,56.9209
1,60.4022,85.6472,88.9381,63.4825,0.143771,104.801,88.542,83.1232,85.4547,72.1856,81.9556,56.9209
2,60.2879,86.4787,86.2512,61.298,0.0780039,106.894,90.3764,84.5464,85.4185,77.4968,84.414,61.4588
3,63.1164,86.2363,91.0983,65.5356,0.133934,110.228,87.4388,87.2588,80.1743,71.4222,81.3226,59.6521
4,60.4022,85.6472,88.9381,63.4825,0.143771,104.801,88.542,83.1232,85.4547,72.1856,81.9556,56.9209


In [327]:
df_final = pd.concat([df_predict, pred], axis = 1)

In [328]:
df_final

,Year,Route,Begin Mile,End Mile,Segment,Maintenance District,Overall Condition Index (OCI),Structural Index,Environmental Index,Average IRI,Rutting Measure,Alligator Cracking WP Index,Edge Cracking Index,Long Cracking Index,Patch Deterioration Index,Raveling and Weathering Index,FW Index,Pavement Condition Rating (PCR)
0,2019,BL-11-M,0.46,2.46,X1,1 - DEMING,60.4022,85.6472,88.9381,63.4825,0.143771,104.801,88.542,83.1232,85.4547,72.1856,81.9556,56.9209
1,2019,BL-11-M,2.46,4.46,X2,1 - DEMING,60.4022,85.6472,88.9381,63.4825,0.143771,104.801,88.542,83.1232,85.4547,72.1856,81.9556,56.9209
2,2019,BL-11-M,4.46,5.42,X3,1 - DEMING,60.2879,86.4787,86.2512,61.298,0.0780039,106.894,90.3764,84.5464,85.4185,77.4968,84.414,61.4588
3,2019,BL-11-P,0.00,0.35,X4,1 - DEMING,63.1164,86.2363,91.0983,65.5356,0.133934,110.228,87.4388,87.2588,80.1743,71.4222,81.3226,59.6521
4,2019,US-84-M,0.00,0.22,X5,2 - ROSWELL,60.4022,85.6472,88.9381,63.4825,0.143771,104.801,88.542,83.1232,85.4547,72.1856,81.9556,56.9209
